In [1]:
!pip install paddlepaddle-gpu
!pip install paddleocr


In [ ]:
# !wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.0g-2ubuntu4_amd64.deb
# !sudo dpkg -i libssl1.1_1.1.0g-2ubuntu4_amd64.deb


In [ ]:
# !git clone https://github.com/PaddlePaddle/PaddleOCR



In [2]:
from paddleocr import PaddleOCR, draw_ocr
from PIL import Image
from IPython import display

In [3]:
! pip install -U spacy -q

In [4]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import spacy
from spacy.tokens import DocBin

In [5]:
import re
import cv2
import pandas as pd
import difflib as diff
from tqdm import tqdm
import pandas as pd


In [7]:
# UPLOAD FILES and UNZIP IT
import zipfile
import os

# Specify the path to the zip file
zip_file_path = '/content/project_files.zip'

# Specify the directory to extract the contents
extract_dir = '/content/'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Folder extracted successfully.")


Folder extracted successfully.


In [8]:
inv_df = pd.DataFrame(columns=[])

In [9]:
image_file = '/content/Images/IMG_0959.jpg'
img = cv2.imread(image_file)

thresh, im_bw = cv2.threshold(img, 110, 200, cv2.THRESH_BINARY)
#inverted_image = cv2.bitwise_not(img)
# ocr = PaddleOCR(lang='en')
ocr = PaddleOCR(use_angle_cls=True)

result = ocr.ocr(img, cls=True)

myresult = ""
for idx in range(len(result)):
    res = result[idx]
    for line in res:
      # text.append(line[1][0])
      myresult +=  f" {line[1][0]}"

download https://paddleocr.bj.bcebos.com/PP-OCRv4/chinese/ch_PP-OCRv4_det_infer.tar to /root/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer/ch_PP-OCRv4_det_infer.tar


100%|██████████| 4.89M/4.89M [00:17<00:00, 283kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/chinese/ch_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer/ch_PP-OCRv4_rec_infer.tar


100%|██████████| 11.0M/11.0M [00:13<00:00, 798kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:07<00:00, 301kiB/s]

[2024/03/09 17:32:54] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='/

[2024/03/09 17:33:01] ppocr DEBUG: dt_boxes num : 20, elapsed : 1.5470526218414307
[2024/03/09 17:33:01] ppocr DEBUG: cls num  : 20, elapsed : 0.18765997886657715
[2024/03/09 17:33:01] ppocr DEBUG: rec_res num  : 20, elapsed : 0.31226563453674316


In [10]:
print(myresult)

 JBC8H0O1 2021 DATE 3 6-20 M.R.P FOR INCL.ALLT K Y PRAX5yPI AX5 JPRAX5 INDIA 9403. E


In [12]:
nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object
nlp_ner = spacy.load("/content/Model/model-best")

In [13]:
# Used Regex

batch_number = re.search(r'(BatchNo,|BatchNo|Batch No.:|Batch No\.|BatchNo\.|Batch|B\.No\.|B\.NO\.|B\.No|BNo.|B.N|N0:|No.|.NO.)\s*(\w+)', myresult)

if batch_number:
    print("Batch number:", batch_number.group(2))
    bno = batch_number.group(2)
else:
    doc = nlp_ner(myresult)
    spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

    for ent in doc.ents:
      if ent.label_ == 'BATCH':
        bno = ent.text
        print(bno)

JBC8H0O1


In [14]:
bno

'JBC8H0O1'

In [15]:
import pandas as pd
from difflib import SequenceMatcher

df = pd.read_csv('/content/Dataset/Medicine_Data.csv')

text_column = df['Batch']

similarity_ratios = []
for i in text_column:
    ratio = SequenceMatcher(None, i, bno).ratio()
    similarity_ratios.append(ratio)

# Find the maximum similarity ratio
max_similarity = max(similarity_ratios)
df['Ratio']=similarity_ratios
# Print the maximum similarity ratio
print("Maximum similarity ratio:", max_similarity)

# saving the dataframe
df.to_csv('/content/Dataset/Similarity_Ratio.csv')

Maximum similarity ratio: 0.875


In [16]:
df = pd.read_csv('/content/Dataset/Similarity_Ratio.csv')
# create a new empty dataframe

max_similarity = max(df['Ratio'])
for index, row in df.iterrows():
    if row['Ratio'] == max_similarity and max_similarity >0:
        #print(row['Company'],row['Name'],row['Batch'],row['MRP'],row['Mfg_dt'],row['Exp_dt'])
        print(row[1:7],"\n")
        Product=row['Name']
        BatchNo=row['Batch']
        Expiry_Date=row['Exp_dt']
        Price=row['MRP']
        inv_df = inv_df.append({'Product': Product,
                           'Batch': BatchNo,
                           'Expiry': Expiry_Date,
                           'Price': Price}, ignore_index=True)

Company    TORRENT PHARMACEUTICALS LTD.
Name                         PRAX 5 TAB
Batch                          JBC8H001
MRP                               151.2
Mfg_dt                             7/21
Exp_dt                             6/23
Name: 80, dtype: object 



In [17]:
inv_df

,Product,Batch,Expiry,Price
0,PRAX 5 TAB,JBC8H001,6/23,151.2
